# **CASE STUDIES FOR DATA ANALYTICS ASSIGNMENT 1**
## **Section 2 : Multi-Source Open Data Quality**
### **By Rishabh Jain(19231092)**


In [2]:
import pandas as pd
import numpy as np
import os
import io
import xml.etree.ElementTree as ET
from pprint import pprint
from opencage.geocoder import OpenCageGeocode

**Data set taken for the assignment:**
1. [DCC] Playing pitches in Dublin City Council - CSV format.
2. [DLR] Playing pitches in DLR - CSV format.
3. [F] Playing Pitches in Fingal - XML format.

**Additional Dataset:**

1. OSiPNG dataset to identify geographical coordinates for some of the locations in other datasets - CSV format.


In [3]:
#LOADING DATASET INTO JUPYTER NOTEBOOK
dcc_csv= pd.read_csv("dccplayingpitchesp20120816-1550.csv")
dlr_csv= pd.read_csv("dlr-pitches.csv")
fcc_xml = open('fccplayingpitchesp20111203-1424.xml').read()
OSi_csv = pd.read_csv("Townlands__OSi_National_Placenames_Gazetteer.csv"
                         , low_memory=False)

XML dataset from FCC needs to be converted to data frame. xml2df function takes XML string and converts it to data frame. 

In [4]:
#Taking code from https://gist.github.com/mattmc3/712f280ec81044ec7bd12a6dda560787
def xml2df(xml_data):
    root = ET.XML(xml_data)
    all_records = []
    for child in root:
        for subchild in child:
            record = {}
            for playing_pitches in subchild:
                record[playing_pitches.tag] = playing_pitches.text
            all_records.append(record)

    df = pd.DataFrame(all_records)
    return df

In [4]:
fcc_xml_raw = xml2df(fcc_xml)

#  Observatory Analysis


## DCC Dataset
Observing the DCC dataset for observation and modelling


In [5]:
dcc_csv.head()

,PARK,AREA,CLUBNAME,LEAGUE,Unnamed: 4
0,ALBERT COLLEGE,NORTH WEST,DRUMCONDRA F.C (Snr),AMATEUR FOOTBALL LEAGUE,NaN
1,ALBERT COLLEGE,NORTH WEST,GLASNEVIN AFC,AMATEUR FOOTBALL LEAGUE,NaN
2,BEECHILL,SOUTH EAST,BALLSBRIDGE FC,AMATEUR FOOTBALL LEAGUE,NaN
3,BELCAMP,NORTH CENTRAL,NEWTOWN CELTIC,AMATEUR FOOTBALL LEAGUE,NaN
4,BELCAMP,NORTH CENTRAL,VIANNEY BOYS,AMATEUR FOOTBALL LEAGUE,NaN


**Observations**
1. There are five columns in the DCC Dataset - *PARK*, *CLUBNAME*, *LEAGUE*, *AREA* and Unnamed.
2. PARK column looks important as it helps us in identification of Parkname of a particular record. CLUBNAME and LEAGUE columns also looks important. AREA column looks redundant as it contains the region location.
3. We will take the column of interest and drop the column which is not required.
4. We can take the required column from the dataset and rename them to match our data model. 


## DLR Dataset
Observing the DLR dataset for observation and modelling

In [6]:
dlr_csv.head(14)

,Location,Number,Size,Latitude,Longitude
0,Kilbogget Park,1,Snr,53.257242,-6.140665
1,NaN,2,SSG,53.257614,-6.139882
2,NaN,3,SSG,53.257842,-6.139265
3,NaN,4,SSG,53.257098,-6.139094
4,NaN,5,SGG,53.256674,-6.140134
5,NaN,6,Snr,53.256597,-6.138149
6,NaN,7,Snr,53.256238,-6.138761
7,NaN,8,Snr,53.256052,-6.139716
8,NaN,9,Snr,53.255667,-6.141805
9,NaN,10,SSG,53.258577,-6.142454


**Observations**

1. The data of interest in DLR dataset is *Location*, *Latitude*, and *Longitude*.

2. There is missing information in the *Location* column, which needs to be replaced with the value from previous row.

3. After that we will take the required columns from the data to make a unified dataset.



## FCC XML Dataset

Observing the FCC dataset for observation and modelling

In [7]:
fcc_xml_raw.head()

,FACILITY_NAME,FACILITY_TYPE,LAT,LOCATION,LONG
0,Balbriggan Town Park,All weather pitches,53.6049596246817,Balbriggan,-6.18235291959051
1,Balheary Reservoir,All weather pitches,53.4727096370551,Swords,-6.22301521551813
2,Town Park,All weather pitches,53.5771135903791,Skerries,-6.11107205744599
3,St. Mologa's Park,All weather pitches,53.6176672458903,Balbriggan,-6.18936794084573
4,Seagrange Park,Basketball Court,53.3966674985382,None,-6.13535180348378


**Observations**

1. The data of interest in FCC dataset is *FACILITY_NAME*, *LAT*, and *LONG*. 

2. The *FACILITY_NAME* can be changed to *Location*.

3. All three columns of interest need to be extracted from the dataset.



## OSiPNG Dataset
Observe OSiPNG dataset and identify data of interest.

In [8]:
OSi_csv.head()

,X,Y,OBJECTID,County,Contae,Local_Government_Area,Limistéar_Rialtas_Áitiúil,Classification,Cineál,Gaeltacht,...,Alternative_Name,IG_E,IG_N,ITM_E,ITM_N,Irish_Validation,Legislation,Validated_By,Date_,Comment_
0,-8.154702,51.812504,10001,CORK,Corcaigh,CORK,NaN,Td,Bf,N,...,NaN,189380,62197,589333,562261,NaN,NaN,Fiontar,NaN,NaN
1,-8.988414,51.890985,10002,CORK,Corcaigh,CORK,NaN,Td,Bf,N,...,NaN,132001,71380,531966,571442,NaN,NaN,Fiontar,NaN,NaN
2,-8.869571,51.678448,10003,CORK,Corcaigh,CORK,NaN,Td,Bf,N,...,NaN,139901,47627,539864,547695,NaN,NaN,Fiontar,NaN,NaN
3,-8.560456,51.990597,10004,CORK,Corcaigh,CORK,NaN,Td,Bf,N,...,NaN,161549,82150,561508,582210,NaN,NaN,Fiontar,NaN,NaN
4,-9.612001,51.612860,10005,CORK,Corcaigh,CORK,NaN,Td,Bf,N,...,NaN,88388,41203,488362,541272,NaN,NaN,Fiontar,NaN,NaN


**Observations**

1. The data of interest in this dataset are *English_Name* and *X* and *Y* columns.

2. *English_Name* can be changed to *Location*.

3. *X* and *Y* contain geographical coordinates (i.e latitutde and longitude).

4. This  this dataset can be potentially used to complete the missing geographical information in the DCC dataset.



# Data Modelling

**Issues in Data Modelling:**

1. Datasets contain same information with different name.

2. No latitude and longitude in DCC.

3. Missing record information in DLR.

4. Extra columns in datasets (need to take the specific column only).

7. Changing the names of columns to location, x and y for our speific format.










**Solution:**

1. Using OSi dataset to get the reuired missing data for DCC dataset.

2. Renaming names of columns to required name. 

3. Copy the previous record to fill the missing data in DLR dataset.

4. Removing the not required columns.

5. Merging the datasets to make required dataset.



## Key Requirement

To make a single dataset containg 3 key items: location, x and y (x and y are the geographical coordinates of the playing pitch).



## Final Data Model


The final dataset will contain following fields

1. Location (object)

2. X (float64)

3. Y (float64)


#  Data Quality Enhancement

## Data Cleaning

1. Extract column of interest from each dataset.

2. Rename columns in each dataset as defined in unified data model.

3. Remove any leading or trailing whitespace characers from string data 

4. Fill in any missing values within columns if possible


### DCC Dataset cleaning

In [9]:
#Extract PARK, CLUBNAME and LEAGUE column, strip any extra whitespaces and renaame
dcc = dcc_csv.PARK.str.strip().to_frame('Location')      
dcc.head()

,Location
0,ALBERT COLLEGE
1,ALBERT COLLEGE
2,BEECHILL
3,BELCAMP
4,BELCAMP


### DLR Dataset cleaning

In [10]:
#Extract columns of interest
dlr = dlr_csv[['Location','Latitude','Longitude']]        
#Remove any whitespaces from Location column
dlr.loc[:,'Location'] = dlr.loc[:,'Location'].str.strip() 
#Rename columns
dlr.rename(columns={'Latitude':'X', 'Longitude':'Y'}, inplace = True) 
dlr.head()

C:\Users\Rishabh Jain\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Rishabh Jain\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,Location,X,Y
0,Kilbogget Park,53.257242,-6.140665
1,NaN,53.257614,-6.139882
2,NaN,53.257842,-6.139265
3,NaN,53.257098,-6.139094
4,NaN,53.256674,-6.140134


In [11]:
#Filling the Location values from previous data where location is NAN.
dlr.loc[:,'Location']=dlr['Location'].ffill()
dlr.head()

,Location,X,Y
0,Kilbogget Park,53.257242,-6.140665
1,Kilbogget Park,53.257614,-6.139882
2,Kilbogget Park,53.257842,-6.139265
3,Kilbogget Park,53.257098,-6.139094
4,Kilbogget Park,53.256674,-6.140134


### FCC Dataset Cleaning

In [12]:
#Extract columns of interest
fcc = fcc_xml_raw[['FACILITY_NAME','LAT','LONG']]          
#Rename columns of interest
fcc.rename(columns={'FACILITY_NAME':'Location','LAT':'X', 
                    'LONG':'Y'}, inplace = True) 
#Remove any whitespaces from Location column
fcc.loc[:,'Location'] = fcc['Location'].str.strip()                                    
fcc.head()

C:\Users\Rishabh Jain\Anaconda3\lib\site-packages\pandas\core\indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


,Location,X,Y
0,Balbriggan Town Park,53.6049596246817,-6.18235291959051
1,Balheary Reservoir,53.4727096370551,-6.22301521551813
2,Town Park,53.5771135903791,-6.11107205744599
3,St. Mologa's Park,53.6176672458903,-6.18936794084573
4,Seagrange Park,53.3966674985382,-6.13535180348378


### OSiPNG Dataset Cleaning

In [13]:
#getting the required columns
OSi = OSi_csv[['English_Name','X','Y']]    
#Rename columns   
OSi.rename(columns={'English_Name':'Location'}, inplace = True)
OSi.loc[:,'Location'] = OSi['Location'].str.strip()               
OSi.head()

,Location,X,Y
0,TULLAHEEN MORE,-8.154702,51.812504
1,TULLATREADA,-8.988414,51.890985
2,TULLIG,-8.869571,51.678448
3,TULLIG,-8.560456,51.990597
4,TULLIG,-9.612001,51.612860


Removing the duplicate data from the datset OSi as this data will be merged with DCC to get the x and y values.

In [14]:
OSi = OSi.drop_duplicates(subset=['Location'])

## Incomplete data challenge 
### Complete DCC using OSI Dataset


In [15]:
DCC_new= pd.merge(dcc[['Location']],OSi[['Location','X','Y']],on='Location',how='left')
DCC_new.head()

,Location,X,Y
0,ALBERT COLLEGE,NaN,NaN
1,ALBERT COLLEGE,NaN,NaN
2,BEECHILL,NaN,NaN
3,BELCAMP,-6.194405,53.408506
4,BELCAMP,-6.194405,53.408506


There are still many missing coordinates for X and Y values. We can resolve this porblem using OpenCageGeoCode



### Using of OpenCageGeoCode to get the required data

We are taking all the location values in DCC and adding Dublin, Ireland to it for geocoder to work. 

We also need to make an account on the website to generate the key


In [16]:
all_locations = dcc['Location'] + ', Dublin, Ireland'
all_locations
np.savetxt("addresses.txt",all_locations, fmt="%s", delimiter="\n")

In [17]:
#Code taken from https://opencagedata.com/tutorials/geocode-in-python
import csv
import itertools
from opencage.geocoder import OpenCageGeocode
from pprint import pprint
key = 'f1a513b5932f4521ae2830da90721485'
geocoder = OpenCageGeocode(key)
addressfile = 'addresses.txt'

try: 
  with open(addressfile,'r+') as f:
    for line in f:
      address = line.strip()
      result = geocoder.geocode(address, no_annotations='1')

      if result and len(result):
        longitude = result[0]['geometry']['lng']
        latitude  = result[0]['geometry']['lat']
        data=[address,latitude,longitude]
        print(data)

      else:
        sys.stderr.write("not found: %s\n" % address)
except IOError:
  print('Error: File %s does not appear to exist.' % addressfile)
except RateLimitExceededError as ex:
  print(ex)
        



['ALBERT COLLEGE, Dublin, Ireland', 53.3854709, -6.2608091]
['ALBERT COLLEGE, Dublin, Ireland', 53.3854709, -6.2608091]
['BEECHILL, Dublin, Ireland', 54.5518653, -5.9180207]
['BELCAMP, Dublin, Ireland', 53.4068274, -6.1969262]
['BELCAMP, Dublin, Ireland', 53.4068274, -6.1969262]
['BELCAMP, Dublin, Ireland', 53.4068274, -6.1969262]
['BELCAMP, Dublin, Ireland', 53.4068274, -6.1969262]
['BELCAMP, Dublin, Ireland', 53.4068274, -6.1969262]
['BELCAMP, Dublin, Ireland', 53.4068274, -6.1969262]
['DARNDALE, Dublin, Ireland', 53.4009603, -6.1940324]
['DONAGHMEDE, Dublin, Ireland', 53.3948612, -6.1635775]
['JOHN PAUL, Dublin, Ireland', 53.3631382, -6.2361848]
['KILMORE, Dublin, Ireland', 54.4097868, -6.6720416]
['KILMORE, Dublin, Ireland', 54.4097868, -6.6720416]
['MARKIEVICZ, Dublin, Ireland', 53.3410089, -6.3400557]
['MOUNT OLIVE, Dublin, Ireland', 53.33306, -6.24889]
['OSCAR TRAYNOR, Dublin, Ireland', 53.3967633, -6.2180479]
['RINGSEND, Dublin, Ireland', 53.341944, -6.226389]
['RINGSEND, Dubli

['WALKINSTOWN, Dublin, Ireland', 53.3216133, -6.3301177]
['ST. ANNES, Dublin, Ireland', 53.3287719, -6.2544648]
['BRICKFIELDS, Dublin, Ireland', 53.3320387, -6.3033135]
['BRICKFIELDS, Dublin, Ireland', 53.3320387, -6.3033135]
['DARTRY, Dublin, Ireland', 53.3099564, -6.2631069]
['FARNHAM, Dublin, Ireland', 53.385891, -6.3003761]
['JOHN PAUL, Dublin, Ireland', 53.3631382, -6.2361848]
['LE FANU, Dublin, Ireland', 53.3358185, -6.3541441]
['RINGSEND, Dublin, Ireland', 53.341944, -6.226389]
['STANNAWAY, Dublin, Ireland', 53.3206866, -6.3006892]
['STANNAWAY, Dublin, Ireland', 53.3206866, -6.3006892]
['STANNAWAY, Dublin, Ireland', 53.3206866, -6.3006892]
['WALKINSTOWN, Dublin, Ireland', 53.3216133, -6.3301177]
['DOLPHIN, Dublin, Ireland', 54.8569788, -6.2698595]
['BEECHILL, Dublin, Ireland', 54.5518653, -5.9180207]
['BRICKFIELDS, Dublin, Ireland', 53.3320387, -6.3033135]
['BRICKFIELDS, Dublin, Ireland', 53.3320387, -6.3033135]
['BRICKFIELDS, Dublin, Ireland', 53.3320387, -6.3033135]
['BUNTING 

Now we are copying all this data into an excel file in the format ["Locations","x","y"] and saving the file as "geo_locations.csv"

After that I am reading the geo_location.csv file


In [1]:
#Reading the geo_location file
geo_locations = pd.read_csv("geo_locations.csv")
geo_locations.head()

NameError: name 'pd' is not defined

Now we are removing the duplicate values from the geolocation dataset

In [19]:
geo_locations = geo_locations.drop_duplicates(subset=['Location'])

Now geo_locations dataset has all geographical coordinates for DCC dataset. Complete dataset can now be created as follows:

In [20]:
dcc.head()

,Location
0,ALBERT COLLEGE
1,ALBERT COLLEGE
2,BEECHILL
3,BELCAMP
4,BELCAMP


In [21]:
dcc_complete= pd.merge(dcc,geo_locations[['Location','X','Y']],on='Location',how='left')
dcc_complete.head()

,Location,X,Y
0,ALBERT COLLEGE,53.385471,-6.260809
1,ALBERT COLLEGE,53.385471,-6.260809
2,BEECHILL,54.551865,-5.918021
3,BELCAMP,53.406827,-6.196926
4,BELCAMP,53.406827,-6.196926


Now we have created our required DCC dataset

In [22]:
result_ds = pd.concat([dcc_complete,fcc,dlr], ignore_index=True, sort=True)
result_ds.head()

,Location,X,Y
0,ALBERT COLLEGE,53.3855,-6.26081
1,ALBERT COLLEGE,53.3855,-6.26081
2,BEECHILL,54.5519,-5.91802
3,BELCAMP,53.4068,-6.19693
4,BELCAMP,53.4068,-6.19693


Saving all the combined dataset to Dublin_Pitches_Unified.csv.

In [23]:
result_ds.to_csv('Dublin_Pitches_Unified.csv',index=False,float_format='%11.5f')

# **APPENDIX**

**1. https://opencagedata.com/tutorials/geocode-in-python**

**2. https://gist.github.com/mattmc3/712f280ec81044ec7bd12a6dda560787**

**3. https://python-forum.io/Thread-Txt-file-to-CSV-file-conversion**

**4. https://medium.com/@robertopreste/from-xml-to-pandas-dataframes-9292980b1c1c**

**5. https://stackoverflow.com/questions/43876246/read-and-process-a-text-file-and-save-to-csv**

**6. https://data.gov.ie/dataset/townlands-osi-national-placenames-gazetteer**

**7. https://data.gov.ie**